# Import

In [121]:
import pandas as pd
import sys
from pathlib import Path
import importlib
# Add the util/input folder to the Python path
util_path = Path("../utils")  
sys.path.append(str(util_path))

# Import the function or module
import visual 
import yield_utils
importlib.reload(visual)
importlib.reload(yield_utils)

<module 'yield_utils' from '..\\utils\\yield_utils.py'>

# Basic Data Understanding
- Remove rows where RunTime, StandByTime, ErrorTime, and StopTime are all 0
1. Basic Information
2. Statistical Summary
3. Check Missing Data
4. Preview first few rows

In [62]:
yield_df = pd.read_csv("../output/yield_data.csv",    
                       parse_dates=['Date'],
                        dtype={
                            'TimeID': 'int32',
                            'RunTime': 'int32',
                            'StandByTime': 'int32',
                            'ErrorTime': 'int32',
                            'StopTime': 'int32',
                            'Yield': 'int32',
                        })
# Remove rows where RunTime, StandByTime, ErrorTime, and StopTime are all 0
yield_df = yield_df[~((yield_df['RunTime'] == 0) & 
                      (yield_df['StandByTime'] == 0) & 
                      (yield_df['ErrorTime'] == 0) & 
                      (yield_df['StopTime'] == 0))].reset_index(drop=True)


In [23]:
visual.df_intro(yield_df)


DataFrame Info:

----------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18945 entries, 0 to 18944
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         18945 non-null  datetime64[ns]
 1   TimeID       18945 non-null  int32         
 2   RunTime      18945 non-null  int32         
 3   StandByTime  18945 non-null  int32         
 4   ErrorTime    18945 non-null  int32         
 5   StopTime     18945 non-null  int32         
 6   Yield        18945 non-null  int32         
dtypes: datetime64[ns](1), int32(6)
memory usage: 592.2 KB
None

Descriptive Statistics:

----------------------------------------
                                Date        TimeID       RunTime  \
count                          18945  18945.000000  18945.000000   
mean   2023-08-30 01:43:40.617577472     12.564001    345.082027   
min              2021-11-16 00:00:00      1.000000      0.

# Data Preparation

## Daily Trend

In [69]:
yield_metrics = ['RunTime', 'StandByTime', 'ErrorTime', 'StopTime', "Yield"]
daily_yield = yield_utils.prepare_yield_data(yield_df, yield_metrics)
daily_yield.head()

Date  RunTime  StandByTime    ErrorTime     StopTime  Yield  \
0 2021-11-16  142.000      152.700   680.400000  2426.300000    8.0   
1 2021-11-17    0.000        0.000  1546.714286  1730.142857    0.0   
2 2021-11-24   51.000        5.625  1248.750000  1326.000000    2.0   
3 2021-12-30    0.000        0.000  3439.166667   139.166667    0.0   
4 2022-01-19   94.125        4.500     6.750000  3033.250000    1.0   

     TotalTime  RunTime_Pct  StandByTime_Pct  ErrorTime_Pct  StopTime_Pct  
0  3401.400000     4.174752         4.489328      20.003528     71.332393  
1  3276.857143     0.000000         0.000000      47.201151     52.798849  
2  2631.375000     1.938150         0.213767      47.456178     50.391905  
3  3578.333333     0.000000         0.000000      96.110852      3.889148  
4  3138.625000     2.998925         0.143375       0.215062     96.642638

## Hourly Trend

In [70]:
hourly_yield = yield_utils.prepare_yield_data(yield_df, yield_metrics, freq='H')
hourly_yield.head()

Date  RunTime  StandByTime  ErrorTime  StopTime  Yield  \
0 2021-11-16 08:00:00      0.0          0.0      432.0    3168.0    0.0   
1 2021-11-16 09:00:00      0.0          0.0     2791.0     809.0    0.0   
2 2021-11-16 10:00:00      0.0          0.0        0.0    3600.0    0.0   
3 2021-11-16 11:00:00      0.0          0.0        0.0    3600.0    0.0   
4 2021-11-16 12:00:00      0.0          0.0        0.0    3600.0    0.0   

   TotalTime  RunTime_Pct  StandByTime_Pct  ErrorTime_Pct  StopTime_Pct  
0     3600.0          0.0              0.0      12.000000     88.000000  
1     3600.0          0.0              0.0      77.527778     22.472222  
2     3600.0          0.0              0.0       0.000000    100.000000  
3     3600.0          0.0              0.0       0.000000    100.000000  
4     3600.0          0.0              0.0       0.000000    100.000000

# Feature Engineer, Split, Train, and Predictions for Yield

## Daily Trend

In [122]:
daily_yield_df = yield_utils.create_yield_features(daily_yield)

# Split data into train, validation, and test sets
X_train_daily, X_val_daily, X_test_daily, y_train_daily, y_val_daily, y_test_daily = yield_utils.split_yield_data(
    daily_yield_df, target='Yield', test_size=0.2, validation_size=0.2
)

# Scale features
X_train_daily_scaled, X_val_daily_scaled, X_test_daily_scaled = yield_utils.scale_features(X_train_daily, X_val_daily, X_test_daily)

# Perform hyperparameter tuning
yield_daily_best_model, yield_daily_best_params = yield_utils.tune_hyperparameters(X_train_daily_scaled, y_train_daily, X_val_daily_scaled, y_val_daily)

# Evaluate the best model on the test set
y_test_daily_pred = yield_daily_best_model.predict(X_test_daily_scaled)
metrics = yield_utils.evaluate_model(y_test_daily, y_test_daily_pred)

# Print metrics
print(metrics)


        Date    RunTime  StandByTime    ErrorTime     StopTime  Yield  \
3 2021-12-30   0.000000     0.000000  3439.166667   139.166667    0.0   
4 2022-01-19  94.125000     4.500000     6.750000  3033.250000    1.0   
5 2022-01-20  46.500000     0.000000     6.000000  2539.000000    0.0   
6 2022-05-25   0.545455     0.000000    23.727273  3575.727273    0.0   
7 2022-05-26  68.526316    28.421053   527.315789  2812.000000    3.0   

     TotalTime  RunTime_Pct  StandByTime_Pct  ErrorTime_Pct  ...  \
3  3578.333333     0.000000         0.000000      96.110852  ...   
4  3138.625000     2.998925         0.143375       0.215062  ...   
5  2591.500000     1.794328         0.000000       0.231526  ...   
6  3600.000000     0.015152         0.000000       0.659091  ...   
7  3436.263158     1.994210         0.827092      15.345617  ...   

   ErrorTime_t-2  StopTime_t-2  Yield_t-2  RunTime_t-3  StandByTime_t-3  \
3    1546.714286   1730.142857        0.0      142.000          152.700   
4 

In [123]:
# Create a comparative DataFrame for the test set
comparison_df = yield_utils.compare_actual_vs_predicted(
    model=yield_daily_best_model,
    X_test=X_test_daily,
    y_test=y_test_daily,
    data=daily_yield_df,
    date_col='Date'
)

# Display the comparison
visual.plot_comparison_with_difference(comparison_df)

## Hourly Trend

# Yield Analysis


In [ ]:
# Productivity vs. Time
yield_df.corr()

ValueError: could not convert string to float: '2021-11-10'

In [ ]:
# Hourly Yield Patterns
yield_df.groupby('Hour')['Yield'].mean().plot(kind='bar')


In [ ]:
# Identify Anomalies

# Error and Downtime Analysis

In [ ]:
# Proportion of Time in Each State
yield_df['ErrorProportion'] = yield_df['ErrorTime'] / 3600
yield_df['StopProportion'] = yield_df['StopTime'] / 3600
yield_df['RunProportion'] = yield_df['RunTime'] / 3600

In [ ]:
# Error Trends
yield_df.groupby('Date')['ErrorTime'].sum().plot(kind='line')

In [ ]:
# Downtime vs. Yield
yield_df.plot.scatter(x='StopTime', y='Yield')
yield_df.plot.scatter(x='ErrorTime', y='Yield')

# Advanced Analysis

## Clustering for Patterns
- Use clustering algorithms like K-Means to group hours/days with similar behavior.

## Anomaly Detection
- Identify unusual hours or days where the distribution of time variables deviates significantly

In [ ]:
from sklearn.ensemble import IsolationForest
model = IsolationForest()
yield_df['Anomaly'] = model.fit_predict(yield_df[['RunTime', 'StandByTime', 'ErrorTime', 'StopTime']])


## Time Series Analysis
- Use rolling averages or decomposition to identify seasonal patterns.

In [ ]:
yield_df['Yield_Rolling'] = yield_df['Yield'].rolling(window=7).mean()

# Visualization

## Heatmap of Time Usage
- Show the distribution of RunTime, StandByTime, etc., across hours and days

In [ ]:
import seaborn as sns
pivot = yield_df.pivot_table(index='Date', columns='Hour', values='RunTime')
sns.heatmap(pivot, cmap='Blues')

## Daily Stacked Bar Chart
* Visualize the proportion of RunTime, StandByTime, etc., for each day

In [ ]:
metrics_to_plot = ['RunTime', 'StandByTime', 'ErrorTime', 'StopTime']
visual.plot_stacked_bar_with_percentage_and_yield(daily_yield, 'Date', metrics_to_plot, 'Yield')